In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 11.5 기간과 기간 연산

In [2]:
# 며칠, 몇 개월, 몇 분기, 몇 해 같은 기간은 Period 클래스로 표현할 수 있으며 문자열이나 정수 그리고 [표 11-4]에서 봤던 빈도를 가지고 생성한다.

In [3]:
p = pd.Period(2007, freq="A-DEC")

In [4]:
p

Period('2007', 'A-DEC')

In [5]:
# 여기서 Period 객체는 2007년 1월 1일부터 같은 해 12월 31일까지의 기간을 표현한다.
# 이 기간에 정수를 더하거나 빼서 편리하게 정해진 빈도에 따라 기간을 이동시킬 수 있다.

In [6]:
p + 5

Period('2012', 'A-DEC')

In [7]:
p - 2

Period('2005', 'A-DEC')

In [8]:
# 만약 두 기간이 같은 빈도를 가진다면 두 기간의 차는 둘 사이의 간격이 된다.

In [9]:
pd.Period("2014", freq="A-DEC") - p

<7 * YearEnds: month=12>

In [10]:
# 일반적인 기간 범위는 period_range 함수로 생성할 수 있다.

In [11]:
rng = pd.period_range("2000-01-01", "2000-06-30", freq="M")

In [12]:
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]')

In [13]:
# PeriodIndex 클래스는 순차적인 기간을 저장하며 다른 pandas 자료구조에서 축 색인과 마찬가지로 사용된다. 

In [14]:
pd.Series(np.random.randn(6), index=rng)

2000-01   -0.204708
2000-02    0.478943
2000-03   -0.519439
2000-04   -0.555730
2000-05    1.965781
2000-06    1.393406
Freq: M, dtype: float64

In [15]:
# 다음과 같은 문자열 배열을 이용해서 PeriodIndex 클래스를 생성하는 것도 가능하다.

In [16]:
values = ["2001Q3", "2002Q2", "2003Q1"]

In [17]:
index = pd.PeriodIndex(values, freq="Q-DEC")

In [18]:
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]')

# 11.5.1 Period의 빈도 변환

In [19]:
# 기간과 PeriodIndex 객체는 asfreq 메서드를 통해 다른 빈도로 변환할 수 있다. 
# 예를 들어 새해 첫날부터 시작하는 연간 빈도를 월간 빈도로 변환해보자. 꽤 간단하게 변환할 수 있다.

In [20]:
p = pd.Period("2007", freq="A-DEC")

In [21]:
p

Period('2007', 'A-DEC')

In [22]:
p.asfreq("M", how="start")

Period('2007-01', 'M')

In [23]:
p.asfreq("M", how="end")

Period('2007-12', 'M')

In [24]:
# Period("2007", "A-DEC")는 전체 기간에 대한 커서로 생각할 수 있고 월간으로 다시 나눌 수 있다.
# 이 내용은 [그림 11-1]을 참조하자(페이지 454). 회계년도 마감이 12월이 아닌 경우에는 월간 빈도가 달라진다.

In [25]:
p = pd.Period("2007", freq="A-JUN")

In [26]:
p

Period('2007', 'A-JUN')

In [27]:
p.asfreq("M", "start")

Period('2006-07', 'M')

In [28]:
p.asfreq("M", "end")

Period('2007-06', 'M')

In [29]:
# 빈도가 상위 단계에서 하위 단계로 변환되는 경우 상위 기간은 하위 기간이 어디에 속했는지에 따라 결정된다.
# 예를 들어 A-JUN 빈도일 경우 2007년 8월은 실제로 2008년 기간에 속하게 된다.

In [30]:
p = pd.Period("Aug-2007", "M")

In [31]:
p.asfreq("A-JUN")

Period('2008', 'A-JUN')

In [32]:
# 모든 PeriodIndex 객체나 시계열은 지금까지 살펴본 내용과 같은 방식으로 변환할 수 있다.

In [33]:
rng = pd.period_range("2006", "2009", freq="A-DEC")

In [34]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [35]:
ts

2006    0.092908
2007    0.281746
2008    0.769023
2009    1.246435
Freq: A-DEC, dtype: float64

In [36]:
ts.asfreq("M", how="start")

2006-01    0.092908
2007-01    0.281746
2008-01    0.769023
2009-01    1.246435
Freq: M, dtype: float64

In [37]:
# 위 예제에서 연 빈도는 해당 빈도의 시작 월부터 시작하는 월 빈도로 치환된다. 
# 만일 매 해의 마지막 영업일을 대신 사용하고 싶다면 "B" 빈도를 사용하고 해당 기간의 종료 지점을 지정해서 변환할 수 있다.

In [38]:
ts.asfreq("B", how="end")

2006-12-29    0.092908
2007-12-31    0.281746
2008-12-31    0.769023
2009-12-31    1.246435
Freq: B, dtype: float64

# 11.5.2 분기 빈도

In [39]:
# 분기 데이터는 재정, 금융 및 다른 분야에서 표준으로 사용된다. 
# 많은 분기 데이터는 일반적으로 회계년도의 끝인 12월의 마지막 날이나 마지막 업무일을 기준으로 보고하는데, 2012Q4는 회계년도의 끝이 어딘가에 따라 의미가 달라진다.

In [40]:
# pandas는 12가지 모든 경우의 수를 지원하며 분기 빈도는 Q-JAN부터 Q-DEC까지다.

In [41]:
p = pd.Period("2012Q4", freq="Q-JAN")

In [42]:
p

Period('2012Q4', 'Q-JAN')

In [43]:
# 회계년도 마감이 1월인 경우라면 2012Q4는 11월부터 1월까지가 되고 일간 빈도로 검사할 수 있다. [그림 11-2]를 살펴보자. 페이지 456

In [44]:
p.asfreq("D", "start")

Period('2011-11-01', 'D')

In [45]:
p.asfreq("D", "end")

Period('2012-01-31', 'D')

In [46]:
# 이렇게 하여 기간 연산을 매우 쉽게 할 수 있는데, 그 예로 분기 영업마감일의 오후 4시를 가리키는 타임스탬프는 다음과 같이 구할 수 있다.

In [47]:
p4pm = (p.asfreq("B", "e") - 1).asfreq("T", "s") + 16 * 60

In [48]:
p4pm

Period('2012-01-30 16:00', 'T')

In [49]:
p4pm.to_timestamp()

Timestamp('2012-01-30 16:00:00')

In [50]:
# period_range를 사용해서 분기 범위를 생성할 수 있다. 연산 역시 동일한 방법으로 수행할 수 있다.

In [51]:
rng = pd.period_range("2011Q3", "2012Q4", freq="Q-JAN")

In [52]:
ts = pd.Series(np.arange(len(rng)), index=rng)

In [53]:
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int32

In [54]:
new_rng = (rng.asfreq("B", "e") - 1).asfreq("T", "s") + 16 * 60

In [55]:
ts.index = new_rng.to_timestamp()

In [56]:
ts

2010-10-28 16:00:00    0
2011-01-28 16:00:00    1
2011-04-28 16:00:00    2
2011-07-28 16:00:00    3
2011-10-28 16:00:00    4
2012-01-30 16:00:00    5
dtype: int32

# 11.5.3 타임스탬프와 기간 서로 변환하기

In [57]:
# 타임스탬프로 색인된 Series와 DataFrame 객체는 to_period 메서드를 사용해서 기간으로 변환 가능하다. 

In [58]:
rng = pd.date_range("2000-01-01", periods=3, freq="M")

In [59]:
ts = pd.Series(np.random.randn(3), index=rng)

In [60]:
ts

2000-01-31    1.007189
2000-02-29   -1.296221
2000-03-31    0.274992
Freq: M, dtype: float64

In [61]:
pts = ts.to_period()

In [62]:
pts

2000-01    1.007189
2000-02   -1.296221
2000-03    0.274992
Freq: M, dtype: float64

In [63]:
# 여기서 말하는 기간은 겹치지 않는 시간상의 간격을 뜻하므로 주어진 빈도에서 타임스탬프는 하나의 기간에만 속한다.
# 새로운 PeriodIndex의 빈도는 기본적으로 타임스탬프 값을 통해 추론되지만 원하는 빈도를 직접 지정할 수도 있다.
# 결과에 중복되는 기간이 나오더라도 문제가 되지 않는다.

In [64]:
rng = pd.date_range("1/29/2000", periods=6, freq="D")

In [65]:
ts2 = pd.Series(np.random.randn(6), index=rng)

In [66]:
ts2

2000-01-29    0.228913
2000-01-30    1.352917
2000-01-31    0.886429
2000-02-01   -2.001637
2000-02-02   -0.371843
2000-02-03    1.669025
Freq: D, dtype: float64

In [67]:
ts2.to_period("M")

2000-01    0.228913
2000-01    1.352917
2000-01    0.886429
2000-02   -2.001637
2000-02   -0.371843
2000-02    1.669025
Freq: M, dtype: float64

In [68]:
# 기간을 타임스탬프로 변환하려면 to_timestamp 메서드를 이용하면 된다. 

In [69]:
pts = ts2.to_period()

In [70]:
pts

2000-01-29    0.228913
2000-01-30    1.352917
2000-01-31    0.886429
2000-02-01   -2.001637
2000-02-02   -0.371843
2000-02-03    1.669025
Freq: D, dtype: float64

In [71]:
pts.to_timestamp(how="end")

2000-01-29 23:59:59.999999999    0.228913
2000-01-30 23:59:59.999999999    1.352917
2000-01-31 23:59:59.999999999    0.886429
2000-02-01 23:59:59.999999999   -2.001637
2000-02-02 23:59:59.999999999   -0.371843
2000-02-03 23:59:59.999999999    1.669025
Freq: D, dtype: float64

# 11.5.4 배열로 PeriodIndex 생성하기

In [72]:
# 고정된 빈도를 갖는 데이터는 종종 여러 컬럼에 걸쳐 기간에 대한 정보를 함께 저장하기도 한다. 
# 예를 들어 다음 거시경제학(매크로경제학) 데이터셋에는 연도와 분기가 구분된 컬럼에 존재한다.

In [74]:
data = pd.read_csv("examples/macrodata.csv")

In [75]:
data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [76]:
data.year

0      1959
1      1959
2      1959
3      1959
4      1960
       ... 
198    2008
199    2008
200    2009
201    2009
202    2009
Name: year, Length: 203, dtype: int64

In [77]:
data.quarter

0      1
1      2
2      3
3      4
4      1
      ..
198    3
199    4
200    1
201    2
202    3
Name: quarter, Length: 203, dtype: int64

In [78]:
# 이 배열을 PeriodIndex에 빈도값과 함께 전달하면 이를 조합해서 DataFrame에서 사용할 수 있는 색인을 만들어낸다.

In [79]:
index = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                       freq="Q-DEC")

In [80]:
index

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203)

In [81]:
data.index = index

In [82]:
data.infl

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
          ... 
2008Q3   -3.16
2008Q4   -8.79
2009Q1    0.94
2009Q2    3.37
2009Q3    3.56
Freq: Q-DEC, Name: infl, Length: 203, dtype: float64